In [70]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("DataLak").getOrCreate()



In [71]:
df=spark.read.csv("file:///home/jovyan/work/data/raw/employee-data.csv",header=True,inferSchema=True)

In [72]:
df.show(3)

+----------+----------+---+------+----------+--------+
|      date|      name| id|salary|department|trim_col|
+----------+----------+---+------+----------+--------+
|2023-01-01|  John Doe|101| 50000|        HR|   abc  |
|2023-01-01|  John Doe|101| 50000|        HR|   abc  |
|2023-02-01|Jane Smith|102| 60000|        IT|      NA|
+----------+----------+---+------+----------+--------+
only showing top 3 rows



In [73]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- trim_col: string (nullable = true)



In [74]:
df.describe()

DataFrame[summary: string, name: string, id: string, salary: string, department: string, trim_col: string]

In [75]:
type(df)

pyspark.sql.dataframe.DataFrame

In [76]:
df.head(2)

[Row(date=datetime.date(2023, 1, 1), name='John Doe', id=101, salary=50000, department='HR', trim_col='  abc  '),
 Row(date=datetime.date(2023, 1, 1), name='John Doe', id=101, salary=50000, department='HR', trim_col='  abc  ')]

In [77]:
type(df.head(2))

list

In [78]:
df_bonus=spark.read.csv("file:///home/jovyan/work/data/raw/dept_bonus.csv",header=True,inferSchema=True)

In [79]:
df_bonus.show(1)

+----------+-----+
|department|bonus|
+----------+-----+
|        HR| 5000|
+----------+-----+
only showing top 1 row



In [80]:
new_df=df.join(df_bonus,"department","left")

In [81]:
new_df.show(3)

+----------+----------+----------+---+------+--------+-----+
|department|      date|      name| id|salary|trim_col|bonus|
+----------+----------+----------+---+------+--------+-----+
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        IT|2023-02-01|Jane Smith|102| 60000|      NA| 7000|
+----------+----------+----------+---+------+--------+-----+
only showing top 3 rows



In [82]:
new_df.printSchema()

root
 |-- department: string (nullable = true)
 |-- date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- trim_col: string (nullable = true)
 |-- bonus: integer (nullable = true)



In [83]:
from pyspark.sql.functions import lit

In [84]:
add_data = [
    ("IT","2024-05-01", "Lisa Ray", 110, 68000, "jkl"),
    ("HR","2024-06-01", "Sam Green", 111, 55000, "NA"),
    ("Sales","2024-07-01", "Kate White", 112, 71000, "mno"),
    ( "Finance","2024-08-01", "Paul Black", 113, 62000, "abc")
]
columns = ["department","date", "name", "id", "salary", "trim_col"]
df_addData=spark.createDataFrame(add_data,columns)



# Add missing 'bonus' column with default value (e.g. 0 or None)
df_addData_with_bonus=df_addData.withColumn("bonus",lit(None).cast("int"))
df_addData_with_bonus.show(2)


new_new_df=new_df.unionByName(df_addData_with_bonus)
new_new_df.show(3)




+----------+----------+---------+---+------+--------+-----+
|department|      date|     name| id|salary|trim_col|bonus|
+----------+----------+---------+---+------+--------+-----+
|        IT|2024-05-01| Lisa Ray|110| 68000|     jkl| NULL|
|        HR|2024-06-01|Sam Green|111| 55000|      NA| NULL|
+----------+----------+---------+---+------+--------+-----+
only showing top 2 rows

+----------+----------+----------+---+------+--------+-----+
|department|      date|      name| id|salary|trim_col|bonus|
+----------+----------+----------+---+------+--------+-----+
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        IT|2023-02-01|Jane Smith|102| 60000|      NA| 7000|
+----------+----------+----------+---+------+--------+-----+
only showing top 3 rows



In [85]:

new_new_df.show(3)

+----------+----------+----------+---+------+--------+-----+
|department|      date|      name| id|salary|trim_col|bonus|
+----------+----------+----------+---+------+--------+-----+
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        HR|2023-01-01|  John Doe|101| 50000|   abc  | 5000|
|        IT|2023-02-01|Jane Smith|102| 60000|      NA| 7000|
+----------+----------+----------+---+------+--------+-----+
only showing top 3 rows



In [86]:
from pyspark.sql.functions import col,sum,avg, trim, upper, when, to_date,coalesce, lit

In [87]:
mean_salary=new_new_df.agg(avg(col("salary"))).alias("mean_salary")
mean_salary.show()


+-----------------+
|      avg(salary)|
+-----------------+
|60428.57142857143|
+-----------------+



In [88]:

mean_salary=mean_salary.collect()[0]["avg(salary)"]

collect turn to list of row 

In [89]:

df_cleaned=(new_new_df.dropna().dropDuplicates(). withColumn("trim_col",trim(col("trim_col")))
                 .withColumn("salary",col("salary").cast("int"))
                  .withColumn("name",upper(col("name")))
                   .withColumn("future_salary",col("salary")*10)
                    .withColumn("trim_col",when(col("trim_col")=="NA",None).otherwise(col("trim_col")))
                    .withColumnRenamed("trim_col","trim")
                     .withColumn("date",to_date(col("date"),"yyy-MM-dd"))
                      .drop("id")
                       .withColumn("salary",when(col("salary")<0,None).otherwise(col("salary")))
                        .withColumn("salary",coalesce(col("salary"),lit(mean_salary)))
           )
                    

                    
           

In [90]:

df_cleaned.show(2)

+----------+----------+----------+-------+----+-----+-------------+
|department|      date|      name| salary|trim|bonus|future_salary|
+----------+----------+----------+-------+----+-----+-------------+
|        HR|2023-01-01|  JOHN DOE|50000.0| abc| 5000|       500000|
|        IT|2023-02-01|JANE SMITH|60000.0|NULL| 7000|       600000|
+----------+----------+----------+-------+----+-----+-------------+
only showing top 2 rows



In [91]:
df_agg=(df_cleaned.filter(col("salary")>60000)
        .groupBy("department").agg(avg("salary"))
       )

In [92]:
df_agg.show()

+----------+-----------+
|department|avg(salary)|
+----------+-----------+
|        HR|    72000.0|
|   Finance|    65000.0|
|        IT|    70000.0|
+----------+-----------+



In [93]:
df_agg.write.mode("overwrite").partitionBy("department").parquet("file:///home/jovyan/work/data/processed/employee_data")

In [95]:
chcek=spark.read.csv("file:///home/jovyan/work/data/processed/employee_data",header=True,inferSchema=True)
